In [2]:
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [4]:
stop_words = ["?", "n't", "'s"]
stop_words += nltk.corpus.stopwords.words("english")

In [5]:
# mounting your google drive to colab
from google.colab import drive

drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [70]:
# load dataset
dataset = r"/gdrive/MyDrive/minor_project_files/filtered.txt"

# df= pd.read_csv(dataset, keep_default_na=False, na_values=['_'])
questions = []
with open(dataset, "r") as fp:
    questions = [x.strip().lower() for x in fp.readlines()]

In [72]:
print(questions[1])

what's it like to work at care.com for your first job?


In [69]:
len(questions)

296559

In [7]:
# embeddings=model(questions[:200000])

In [25]:
# sem = np.asarray(sem)
# filename1="sent_enc_embeddings1"
# np.save(f'/gdrive/MyDrive/minor_project_files/{filename1}.npy', embeddings)

In [11]:
# embeddings2=model(questions[200000:])

In [24]:
# embeddings2.shape

TensorShape([96559, 512])

In [26]:
# # sem = np.asarray(sem)
# filename2="sent_enc_embeddings2"
# np.save(f'/gdrive/MyDrive/minor_project_files/{filename2}.npy', embeddings2)

In [27]:
# loaded_embeddings1 = np.load(f'/gdrive/MyDrive/minor_project_files/{filename1}.npy', allow_pickle=True)
# # print(sem[:1])
# print(loaded_embeddings1.shape)

(200000, 512)


In [28]:
# loaded_embeddings2 = np.load(f'/gdrive/MyDrive/minor_project_files/{filename2}.npy', allow_pickle=True)
# # print(sem[:1])
# print(loaded_embeddings2.shape)

(96559, 512)


In [31]:
# full_embeddings_filename="sentence_encoder_embeddings"
# full_embeddings=np.concatenate((loaded_embeddings1,loaded_embeddings2),axis=0)
# np.save(f'/gdrive/MyDrive/minor_project_files/{full_embeddings_filename}.npy', full_embeddings)

In [35]:
sent_enc_embeddings = np.load(
    f"/gdrive/MyDrive/minor_project_files/{full_embeddings_filename}.npy",
    allow_pickle=True,
)

In [32]:
!pip install annoy

     |████████████████████████████████| 646 kB 5.0 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391671 sha256=94d3a0da057b5ee4e883d0f2ec70af103597f6500facdc3b1ceb9eca4ac6f026
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [33]:
import annoy
import pickle


class AnnoyIndex:
    def __init__(self, dimension):
        self.dimension = dimension
        self.index = annoy.AnnoyIndex(self.dimension)

    def build(self, vectors, labels, number_of_trees=5):
        self.vectors = vectors
        self.labels = labels

        for i, vec in enumerate(self.vectors):
            if not np.isnan(np.sum(vec)):
                self.index.add_item(i, vec)
        self.index.build(number_of_trees)

    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(list(vector), k)
        return [self.labels[i] for i in indices]

    def save(self, path):
        label_path = path.split(".")[0] + ".labels"
        print(label_path)
        with open(label_path, "wb") as fp:
            pickle.dump(self.labels, fp)
        self.index.save(path)

    def load(self, path):
        label_path = path.split(".")[0] + ".labels"
        self.index = annoy.AnnoyIndex(self.dimension)
        with open(label_path, "rb") as fp:
            self.labels = pickle.load(fp)
        self.index.load(path)

In [34]:
real_questions = []
with open(dataset, "r") as fp:
    real_questions = fp.readlines()

In [36]:
# create annoy index from vectors
index = AnnoyIndex(dimension=len(sent_enc_embeddings[0]))
index.build(sent_enc_embeddings, real_questions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


In [38]:
sent_enc_index = "sent_enc_index"
# index.save(f'/gdrive/MyDrive/minor_project_files/{sent_enc_index}.ann')

/gdrive/MyDrive/minor_project_files/sent_enc_index.labels


In [39]:
#  load existing annoy index from file
loaded_index = AnnoyIndex(dimension=len(sent_enc_embeddings[0]))
loaded_index.load(f"/gdrive/MyDrive/minor_project_files/{sent_enc_index}.ann")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.


In [66]:
loaded_index.query(sent_enc_embeddings[1])
# print(real_questions[1])
# print(sem[1])

["What's it like to work at Care.com for your first job?\n",
 "What's it like to work at Ingredion for your first job?\n",
 "What's it like to work at Balchem for your first job?\n",
 "What's it like to work at HFF for your first job?\n",
 "What's it like to work at EMCORE for your first job?\n",
 "What's it like to work at Middleby for your first job?\n",
 "What's it like to work at ParkOhio for your first job?\n",
 "What's it like to work at Stamps.com for your first job?\n",
 "What's it like to work at E.W. Scripps for your first job?\n",
 "What's it like to work at Knoll for your first job?\n"]

In [44]:
def embed(input):
    return model(input)

In [73]:
question = "what's it like to work at care.com for your first job?"
emb = embed([question])
loaded_index.query(embed([question])[0])

["What's it like to work at Care.com for your first job?\n",
 "What's it like to work at Ingredion for your first job?\n",
 "What's it like to work at Balchem for your first job?\n",
 "What's it like to work at HFF for your first job?\n",
 "What's it like to work at EMCORE for your first job?\n",
 "What's it like to work at Middleby for your first job?\n",
 "What's it like to work at ParkOhio for your first job?\n",
 "What's it like to work at Stamps.com for your first job?\n",
 "What's it like to work at E.W. Scripps for your first job?\n",
 "What's it like to work at Knoll for your first job?\n"]

In [80]:
def get_similar(input_question: str):
    emb = embed([input_question])
    return loaded_index.query(emb[0])

In [82]:
get_similar("why is alan turing considered father of computer science?")

['What does a computer science engineer does?\n',
 'When did Alan Turing receive recognition for the Enigma?\n',
 'Does how much you know in computer science mean your a better programmer?\n',
 'If all computers suddenly failed, what would happen to people with computer science degrees?\n',
 'How do you justify computer programming as engineering?\n',
 'What does it take to be a programmer?\n',
 'What is the most frustrating thing about being a computer programmer?\n',
 'Why do most people think that the computer science engineers are better than the mechanical engineers?\n',
 'If a scientist, an engineer, a metallurgist, a physicist, and a programmer were sent back in time, how much time would it take to build a computer?\n',
 'Why are graduates in Mathematics hired for software engineering job?\n']